In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import imsave
from PIL import Image  #注意Image,后面会用到

import os 

import threading
import time

In [ ]:
def unpickle(filename):
    import pickle
    with open(filename, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

data_path = "datasets/cifar10/"

meta = unpickle(data_path + '/batches.meta')
label_name = meta[b'label_names']

for i in range(1,6):
    content = unpickle(data_path + '/data_batch_' + str(i))
    print('loading data...')
    print(content.keys())
    print('transforming data_batch' + str(i))
    for j in range(10000):
        img = content[b'data'][j]
        img = img.reshape(3,32,32)
        img = img.transpose(1,2,0)
        
        img_path = data_path + 'train/'+label_name[content[b'labels'][j]].decode()
        if not os.path.isdir(img_path):
            os.makedirs(img_path)
            
        img_name = img_path + '/batch_' + str(i) + '_num_' + str(j) +'.jpg'
        
        imsave(img_name,img)

## test data
content = unpickle(data_path + 'test_batch')
print('loading data...')
print(content.keys())
print('transforming test_batch')
for j in range(10000):
    img = content[b'data'][j]
    img = img.reshape(3,32,32)
    img = img.transpose(1,2,0)
        
    img_path = data_path + 'test/'+label_name[content[b'labels'][j]].decode()
    if not os.path.isdir(img_path):
        os.makedirs(img_path)
            
    img_name = img_path + '/num_' +str(j) +'.jpg'
        
    imsave(img_name,img)

## TFRecords
TensorFlow可以支持cifar10的数据格式， 也提供了标准的TFRecord 格式，而关于 tensorflow 读取数据， 官网提供了3中方法 
1 Feeding： 在tensorflow程序运行的每一步， 用python代码在线提供数据 
2 Reader ： 在一个计算图（tf.graph）的开始前，将文件读入到流（queue）中 
3 在声明tf.variable变量或numpy数组时保存数据。受限于内存大小，适用于数据较小的情况

在本文，主要介绍第二种方法，利用tf.record标准接口来读入文件,训练时不再使用feed,而是事先将数据的读取也构建成图，将数据读取图的输出当作训练输入直接与训练图拼接再一起，不需要再使用tf.palceholder,这样训练时tf将直接从硬盘中读取数据

tfrecord, 这是一种将图像数据和标签放在一起的二进制文件，能更好的利用内存，在tensorflow中快速的复制，移动，读取，存储 等等..
tf.train.Example 协议内存块包含了Features字段，通过feature将图片的二进制数据和label进行统一封装， 然后将example协议内存块转化为字符串， tf.python_io.TFRecordWriter 写入到TFRecords文件中

In [ ]:
data_path_train = "datasets/cifar10/train/"
classes = ['airplane','automobile', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 

filename = data_path_train + "cifar_train.tfrecords"
writer= tf.python_io.TFRecordWriter(filename) #要生成的文件

for index in range(len(classes)):
    class_path = data_path_train + classes[index] +'/'
    
    for img_name in os.listdir(class_path): 
        img_path=class_path+img_name #每一个图片的地址

        img=Image.open(img_path)
        #img= img.resize((128,128))
        img_raw=img.tobytes()#将图片转化为二进制格式
        example = tf.train.Example(features=tf.train.Features(feature={
            "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
            'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
        })) #example对象对label和image数据进行封装
        writer.write(example.SerializeToString())  #序列化为字符串

writer.close()

In [ ]:
data_path_test = "datasets/cifar10/test/"
classes = ['airplane','automobile', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 

filename = data_path_test + "cifar_test.tfrecords"
writer= tf.python_io.TFRecordWriter(filename) #要生成的文件

for index in range(len(classes)):
    class_path = data_path_test + classes[index] +'/'
    
    for img_name in os.listdir(class_path): 
        img_path=class_path+img_name #每一个图片的地址

        img=Image.open(img_path)
        #img= img.resize((128,128))
        img_raw=img.tobytes()#将图片转化为二进制格式
        example = tf.train.Example(features=tf.train.Features(feature={
            "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
            'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
        })) #example对象对label和image数据进行封装
        writer.write(example.SerializeToString())  #序列化为字符串

writer.close()

## 读取TFRECORD文件

In [ ]:
def read_and_decode(filename): # 读入dog_train.tfrecords
    filename_queue = tf.train.string_input_producer([filename])#生成一个queue队列

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)#返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })#将image数据和label取出来

    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.reshape(img, [32, 32, 3])  #reshape为32*32的3通道图片
    img = tf.cast(img, tf.float32) * (1. / 255) - 0.5 #在流中抛出img张量
    label = tf.cast(features['label'], tf.int32) #在流中抛出label张量
    return img, label


In [ ]:
## 注意，feature的属性“label”和“img_raw”名称要和制作时统一
## 返回的img数据和label数据一一对应。返回的img和label是2个 tf 张量
'''
(<tf.Tensor 'sub:0' shape=(32, 32, 3) dtype=float32>,
 <tf.Tensor 'Cast_1:0' shape=() dtype=int32>)
'''

read_and_decode(filename)

## 显示tfrecord格式的图片

In [ ]:
data_path_train = "datasets/cifar10/train/"

filename_train = data_path_train + "cifar_train.tfrecords"

filename_queue = tf.train.string_input_producer([filename_train]) #读入流中
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)   #返回文件名和文件
features = tf.parse_single_example(serialized_example,
                                   features={
                                       'label': tf.FixedLenFeature([], tf.int64),
                                       'img_raw' : tf.FixedLenFeature([], tf.string),
                                   })  #取出包含image和label的feature对象

image = tf.decode_raw(features['img_raw'], tf.uint8)
image = tf.reshape(image, [32, 32, 3])
label = tf.cast(features['label'], tf.int32)

with tf.Session() as sess: #开始一个会话
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    ## 
    coord=tf.train.Coordinator()
    threads= tf.train.start_queue_runners(coord=coord)
    for i in range(10): ##  不使用epoch, 手动控制循环数量
        example, l = sess.run([image,label])#在会话中取出image和label
        img=Image.fromarray(example, 'RGB')#这里Image是之前提到的
        img.save(data_path_train + str(i)+'_''Label_'+str(l)+'.jpg')#存下图片
        #print(example, l)
    ##     
    coord.request_stop()
    coord.join(threads)

## 使用TFrecod 数据训练
（将以上代码清空输出）

In [6]:
import tensorflow as tf
import numpy as np
import threading

num_epochs = 50
batch_size =100
data_path_train = "datasets/cifar10/train/"

filename_train = data_path_train + "cifar_train.tfrecords"

filename_train_queue = tf.train.string_input_producer(
    [filename_train], num_epochs=num_epochs) #读入流中, 并设定epoch数量

reader_train = tf.TFRecordReader()

_, serialized_example_train = reader_train.read(filename_train_queue)   #返回文件名和文件

features_train = tf.parse_single_example(serialized_example_train,
                                   features={
                                       'label': tf.FixedLenFeature([], tf.int64),
                                       'img_raw' : tf.FixedLenFeature([], tf.string),
                                   })  #取出包含image和label的feature对象

image_train = tf.decode_raw(features_train['img_raw'], tf.uint8)
image_train = tf.reshape(image_train, [32, 32, 3])
image_train = tf.cast(image_train, tf.float32)*(1./255) - 0.5
label_train = tf.cast(features_train['label'], tf.int32)

images_batch_train, labels_batch_train = tf.train.shuffle_batch(
    [image_train, label_train], batch_size=batch_size, 
    capacity=2000, 
    min_after_dequeue=1000)


In [3]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

def conv2d(x,W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x): 
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

def conv_layer(input, shape):
    W = weight_variable(shape)
    b = bias_variable([shape[3]]) # number of filters
    return tf.nn.relu(conv2d(input,W) + b)

def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size,size])
    b = bias_variable([size])
    return tf.matmul(input,W) + b

In [7]:
x = tf.placeholder(tf.float32, [batch_size,32,32,3])
y = tf.placeholder(tf.float32, [batch_size,10])
keep_prob = tf.placeholder(tf.float32)
    
conv1_1 = conv_layer(x,shape=[3,3,3,32])
conv1_2 = conv_layer(conv1_1,shape=[3,3,32,32])
conv1_3 = conv_layer(conv1_2,shape=[3,3,32,32])
conv1_pool = max_pool_2x2(conv1_3) # 16,32
conv1_drop = tf.nn.dropout(conv1_pool,keep_prob=keep_prob)
    
conv2_1 = conv_layer(conv1_drop,shape=[3,3,32,64])
conv2_2 = conv_layer(conv2_1,shape=[3,3,64,64])
conv2_3 = conv_layer(conv2_2,shape=[3,3,64,64])
conv2_pool = max_pool_2x2(conv2_3) # 8,64
conv2_drop = tf.nn.dropout(conv2_pool,keep_prob=keep_prob)
    
conv3_1 = conv_layer(conv2_drop,shape=[3,3,64,128])
conv3_2 = conv_layer(conv3_1,shape=[3,3,128,128])
conv3_3 = conv_layer(conv3_2,shape=[3,3,128,128])
conv3_pool = tf.nn.max_pool(conv3_3, ksize=(1,8,8,1), strides=(1,8,8,1), padding='SAME') # 1,128
conv3_flat = tf.reshape(conv3_pool, [-1,128])
conv3_drop = tf.nn.dropout(conv3_flat,keep_prob=keep_prob)
    
full1 = tf.nn.relu(full_layer(conv3_drop, 600))
full1_drop = tf.nn.dropout(full1, keep_prob=keep_prob)
    
yout = full_layer(full1_drop,10)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=yout,labels=y))

learning_rate = 5e-4
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(yout,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [9]:
with tf.Session() as sess:
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)
        
    ##### coordinator ####
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    try:
        step = 0
        ## 事先已设定了epoch,不再需要控制循环次数
        while not coord.should_stop():
            step += 1
            image_batch, label_batch = sess.run([images_batch_train, tf.one_hot(labels_batch_train,depth=10)])
        
            feed = {x:image_batch, y:label_batch, keep_prob:0.5}
            # loss,_,acc=session.run([cross_entropy,train_step,accuracy], feed_dict=feed)
            sess.run(train_step, feed_dict=feed, )
        
            if step%100 == 0:
                loss = sess.run(cross_entropy, feed_dict=feed)
                train_accuracy = sess.run(accuracy, feed_dict={x:image_batch, y:label_batch, keep_prob:1.0})
                print("step {} loss: {}, train_acc: {:.4}%".format(step, loss, train_accuracy*100))
        
            if step>1500:
                learning_rate = learning_rate * 0.5
        
    except tf.errors.OutOfRangeError:
        print('Done training for %d epochs, %d steps.' % (num_epochs, step))
    
    finally:
        # when done, ask the treads to stop.
        coord.request_stop()
    ## wait for threads to finish
    coord.join(threads)
  
        
        
        
        
    

KeyboardInterrupt: 

In [ ]:
num_epochs = 10
batch_size =100
data_path = "datasets/cifar10/test/"

filename = data_path + "cifar_test.tfrecords"

filename_train_queue = tf.train.string_input_producer(
    [filename_train], num_epochs=num_epochs) #读入流中, 并设定epoch数量

reader_train = tf.TFRecordReader()

_, serialized_example_train = reader_train.read(filename_train_queue)   #返回文件名和文件

features_train = tf.parse_single_example(serialized_example_train,
                                   features={
                                       'label': tf.FixedLenFeature([], tf.int64),
                                       'img_raw' : tf.FixedLenFeature([], tf.string),
                                   })  #取出包含image和label的feature对象

image_train = tf.decode_raw(features_train['img_raw'], tf.uint8)
image_train = tf.reshape(image_train, [32, 32, 3])
image_train = tf.cast(image_train, tf.float32)*(1./255) - 0.5
label_train = tf.cast(features_train['label'], tf.int32)

images_batch_train, labels_batch_train = tf.train.shuffle_batch(
    [image_train, label_train], batch_size=batch_size, 
    capacity=2000, 
    min_after_dequeue=1000)
        
        X = cifar.test.images.reshape(10,1000,32,32,3)
        Y = cifar.test.labels.reshape(10,1000,10)
        
        test_accuracy = np.mean([sess.run(accuracy, feed_dict={x:X[i], y:Y[i], keep_prob:1.0}) for i in range(10)])
        print("test_acc: {}".format(test_accuracy))
            